In [1]:
import numpy as np
from pytimedinput import timedInput

In [3]:
ITERATIONS = 35
MEAN_MIN = 100
MEAN_MAX = 250

STD_MIN = 5
STD_MAX = 50
STD_SETTLEMENT = 25

CROSS_PROB = 0.4

TIME_LOW = 0.2
TIME_HIGH = 2

Bid = 1
Offer = 0
Buy = 1
Sell = -1
side_map = {1: "Bid", 0: "Offer"}

class Quote:
    def __init__(self, price, side) -> None:
        self.price = round(price)
        self.side = side  # 1 = bid, # 0 = offer

    def __repr__(self):
        return f"{side_map[self.side]} {self.price}"


class Book:
    def __init__(self, title='A'):
        self.bids = []
        self.offers = []
        self.title = title

    def best_offer(self):
        return min(self.offers) if self.offers else 2e16

    def best_bid(self):
        return max(self.bids) if self.bids else -2e16

    def display(self):
        bids = sorted(self.bids, reverse=True)
        offers = sorted(self.offers, reverse=True)

        print(f'Book: {self.title}')
        print('-'*20)
        for o in offers:
            print(f'     | {o} ')
        for b in bids:
            print(f' {b} |  ')

    def clean_book(self):
        if len(self.offers) > 5:
            worst_offer = max(self.offers)
            self.offers.remove(worst_offer)
            print(f"Removed {worst_offer} Offer")
        if len(self.bids) > 5:
            worst_bid = min(self.bids)
            self.bids.remove(worst_bid)
            print(f"Removed {worst_bid} Bid")

    def append(self, quote):
        # Appends to order book (quote is not in cross)
        print(quote)
        if quote.side:
            self.bids.append(quote.price)
        else:
            self.offers.append(quote.price)

        self.clean_book()

    def process_quote(self, quote):
        # Quote is in cross
        lift = quote.side == Bid and quote.price > self.best_offer()
        hit = quote.side == Offer and quote.price < self.best_bid()

        if lift:
            print(f"{self.best_offer()} Offer Lifted")
            self.offers.remove(self.best_offer())
        elif hit:
            print(f"{self.best_bid()} Bid Hit")
            self.bids.remove(self.best_bid())
        else:
            self.append(quote)


class Bot:
    def __init__(self):
        self.theo = np.random.uniform(MEAN_MIN, MEAN_MAX)

    def calc_var(self, i):
        var_width = STD_MAX - STD_MIN
        dec_per_it = var_width / ITERATIONS

        return STD_MAX - i*dec_per_it

    def quote(self, i):
        # bid = 1
        price = np.random.normal(self.theo, self.calc_var(i))
        cross = np.random.random() < CROSS_PROB # the bot will cross itself (quote a bad price)

        if price < self.theo:
            if cross:
                side = Offer
            else:
                side = Bid
        else:
            if cross:
                side = Bid
            else:
                side = Offer
        # side = int(price < self.theo and not cross)

        return Quote(price, side)


class Game:
    def __init__(self):
        self.bot = Bot()
        self.book = Book()
        self.position = 0
        self.settlement = np.random.normal(self.bot.theo, STD_SETTLEMENT)
        self.trades = []

    def start(self):
        for i in range(ITERATIONS):
            q = self.bot.quote(i)
            self.book.process_quote(q)
            player_action, missed = timedInput(timeout=1, resetOnInput=False, endCharacters='\r')

            if missed:
                if player_action:
                    print('No Action')
            else:
                self.process_action(player_action)

            self.book.display()
            variable_speed = int(round(np.random.uniform(TIME_LOW,TIME_HIGH)))
            player_action, missed = timedInput(timeout=variable_speed, resetOnInput=False, endCharacters='\r')

            if missed:
                if player_action:
                    print('No Action')
            else:
                self.process_action(player_action)

            print("-"*80)

        print(f"Position: {self.position}")
        print(f"Trades: {self.trades}")
        print(f'Settles @ {self.settlement}')
        print(f'Bot Theo: {self.bot.theo}')
        print(f'PnL: $ {round(self.settlement*self.position - sum(self.trades), 2)}')

    def process_action(self, action):
        """Action must be hit, lift, mine or yours"""
        if action in ['h']:
            self.execute(Sell, self.book.best_bid())

        if action in ['l']:
            self.execute(Buy, self.book.best_offer())

    def execute(self, side, price):
        self.position += side
        self.trades.append(side * price)
        if side > 0:
            print(f"Bought @ {self.book.best_offer()}!")
            self.book.offers.remove(self.book.best_offer())
        else:
            print(f"Sold @ {self.book.best_bid()}!")
            self.book.bids.remove(self.book.best_bid())


# if __name__ == '__main__':
#     g = Game()
#     g.start()

In [58]:
import numpy as np

np.random.choice([1,53,14,5,6], 3, replace=False)

array([1, 6, 5])

In [36]:
import pandas as pd
lst = [np.random.exponential(scale=2) for _ in range(100000)]
pd.Series(np.array([np.ceil(i) for i in lst])).value_counts(normalize=True)

1.0     0.39436
2.0     0.23809
3.0     0.14638
4.0     0.08610
5.0     0.05298
6.0     0.03223
7.0     0.01929
8.0     0.01170
9.0     0.00747
10.0    0.00439
11.0    0.00282
12.0    0.00167
13.0    0.00100
14.0    0.00067
15.0    0.00040
16.0    0.00014
17.0    0.00014
19.0    0.00006
18.0    0.00004
21.0    0.00003
25.0    0.00001
24.0    0.00001
22.0    0.00001
20.0    0.00001
dtype: float64

In [63]:
import plotly.express as px

[(action, book_label) for action, book_label in ['ha', 'lb', 'ho']]

[('h', 'a'), ('l', 'b'), ('h', 'o')]

In [67]:
np.sign(0)

0

In [28]:
fig = px.histogram(lst)
fig.show()